In [ ]:
#load python packages
import os
import io
import csv
import zipfile

In [ ]:
#Import the Wedge Zipfiles
zip_files = os.listdir("WedgeZipFiles/ZipFiles/")
from zipfile import ZipFile

In [ ]:
# Check wedge data for delimiters and delimiter type
delimiters = dict()

# Start by reading in all the files again.

for this_zf in zip_files :
    with ZipFile("WedgeZipFiles/ZipFiles/" + this_zf,'r') as zf :
        zipped_files = zf.namelist()

        for file_name in zipped_files :
            input_file = zf.open(file_name,'r')
            input_file = io.TextIOWrapper(input_file,encoding="utf-8")
            
            dialect = csv.Sniffer().sniff(sample=input_file.readline(),
                                      delimiters=[",",";","\t"])
            
            delimiters[file_name] = dialect.delimiter
            
            #input_file.seek(0)
            
            print(" ".join(["It looks like",
                           file_name,
                           "has delimiter",
                           dialect.delimiter,
                           "."]))

            input_file.close() # tidy up

In [ ]:
# store the Wedge headers
headers = ["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"]

In [ ]:
# this section does the work of cleaning and storing the cleaned wedge data with consistent headers in the temp folder temp_clean_lg
if not os.path.isdir("temp_clean_lg") : # if folder exists
    os.mkdir("temp_clean_lg")           # if not, make it

for zip_file in zip_files : # start the loop for data cleaning
    
    with ZipFile("WedgeZipFiles/ZipFiles/" + zip_file,'r') as my_zip_file : 
    
        files_inside = my_zip_file.namelist()
        for zipped_file in files_inside :
            sniffer = csv.Sniffer()

            print(f"Processing {zipped_file} now.") # print which file is being worked on to monitor status
            
            with my_zip_file.open(zipped_file,'r') as input_file :
                
                output_file_name = input_file.name.replace(".csv","_clean.csv") # output the cleaned wedge csv file into the temp folder with the *_clean.csv name

                with open("temp_clean_lg/" + output_file_name,'w') as outfile : # join the comma deliniated table with the stored header
                    outfile.write(",".join(headers) + "\n")
                                    
                    rows_printed = 0
                    for idx, line in enumerate(input_file) :
        
                        file_has_header = False # set the header as false
                        
                        # clean the data removing null values check the delimer and make delimiters consistent as comma deliniated
                        dialect = sniffer.sniff(line.decode("utf-8"))
                        line = line.decode("utf-8").strip().split(dialect.delimiter)
                        line = [piece.replace('"','') for piece in line]
                        line = [piece.replace('\\N','') for piece in line]
                        line = [piece.replace('NULL','') for piece in line]
                        
                        # there is a data description yogurt, granola, fruit which if left with the comma deliniation creates too many columns
                        # this code finds those columns and removes the commas from between these values and concatinates yogurt fruit and granola into the description field
                        if len(line) != 50:
                            new_column = (''. join(line [5:8]))
                            del line[5:8]
                            line.insert(5,new_column)
                        
                        # if the first string item in the csv has the word datetime then headers are present and set the header check object to true
                        if idx == 0 :
                            if 'datetime' in line[0] :
                                file_has_header = True
                        
                        # if header is true then pass over the .csv file
                        if file_has_header and idx == 0 :
                            # don't print line
                            pass
                        
                        # if there is no header then join the header to the data in the .csv
                        else : 
                            outfile.write(",".join(line) + "\n")
                            rows_printed += 1